## Linear Regression: Best Subset and Stepwise Regression

**Functions**

`np.linalg.lstsq`, `sklearn.model_selection.cross_val_score`, `sklearn.linear_model.LinearRegression`, `sklearn.model_selection.KFold`

In [1]:
import numpy as np
import pandas as pd

### Exercise 41
Download data from [Ken French's website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/) on the VWM and the 12 industry portfolios. Reformat both data sets so that they has a `DatetimeIndex`.


In [2]:
vwm = pd.read_csv("data/VWM.csv",index_col="Date")
vwm.index = pd.to_datetime(vwm.index, format="%Y%m")
vwm = vwm.resample("M").last()

industries = pd.read_csv("data/12_Industry_portfolios.csv",index_col="Date")
industries.index = pd.to_datetime(industries.index,format="%Y%m")
industries = industries.resample("M").last()
industries

,NoDur,Durbl,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
Date,,,,,,,,,,,,
1926-07-31,1.45,15.55,3.67,-1.18,8.01,3.16,0.83,7.04,0.11,1.77,0.37,2.24
1926-08-31,3.97,3.68,2.42,3.47,5.14,1.97,2.17,-1.69,-0.71,4.25,4.46,4.37
1926-09-30,1.14,4.80,-0.07,-3.39,5.30,-0.34,2.41,2.04,0.21,0.69,-1.23,0.35
1926-10-31,-1.24,-8.23,-3.16,-0.78,-4.55,-5.38,-0.11,-2.63,-2.29,-0.57,-5.16,-2.76
1926-11-30,5.20,-0.19,3.82,0.01,5.11,4.79,1.63,3.71,6.43,5.42,2.24,2.10
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-30,-0.03,14.20,3.31,-0.36,1.23,6.06,-2.44,-5.04,4.21,-1.76,-0.35,0.45
2020-07-31,5.85,18.73,2.90,-4.77,7.48,6.87,5.09,6.37,9.53,4.44,1.96,6.22
2020-08-31,4.43,40.86,6.98,-1.14,4.96,10.67,5.45,-2.25,8.13,2.77,4.97,10.16


### Exercise 42

Use Best Subset Regression with cross-validation to select the weights of a tracking portfolio where the industry portfolios are used to track the value-weighted-market. Use data until the end of 2014. A tracking portfolio is a portfolio that replicates a portfolio using other assets. The weights can be estimated using a regression model excludes a constant.

$$ R_{i,p} = \beta_1 R_{i,1} + \beta_2 R_{i,2} + \ldots + \beta_k R_{i,k} + \epsilon_{i} $$

where $R_{i,j}$ are returns on the assets used to construct the tracking portfolio. The constant is excluded since the portfolio should track both the shock and match the mean return. OLS minimizes the variance of the tracking error (in-sample).

In [3]:
x = industries["1980":"2014"]
y = vwm["VWM"]["1980":"2014"]
t, p = x.shape

In [4]:
from itertools import combinations

for i in range(1, p+1):
    count = 0
    for comb in combinations(x.columns, i):
        print(comb)
        count += 1
        if count > 1:
            break

('NoDur',)
('Durbl',)
('NoDur', 'Durbl')
('NoDur', 'Manuf')
('NoDur', 'Durbl', 'Manuf')
('NoDur', 'Durbl', 'Enrgy')
('NoDur', 'Durbl', 'Manuf', 'Enrgy')
('NoDur', 'Durbl', 'Manuf', 'Chems')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'BusEq')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'Telcm')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Utils')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Shops')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Hlth ')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops', 'Hlth ')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops

In [5]:
from numpy.linalg import lstsq

best_models = {}
for i in range( 1, p+1):
    best_sse = np.inf
    for comb in combinations(x.columns, i):
        reg = x[list(comb)]
        beta = lstsq(reg, y , rcond=None)[0]
        resid = y - reg @ beta
        sse = resid @ resid
        if sse < best_sse:
            best_sse = sse
            best_models[i] = list(comb)
pd.Series(best_models)


1                                               [Other]
2                                        [BusEq, Money]
3                                 [NoDur, BusEq, Other]
4                          [NoDur, Enrgy, BusEq, Money]
5                   [NoDur, Enrgy, BusEq, Money, Other]
6            [NoDur, Enrgy, BusEq, Telcm, Money, Other]
7     [NoDur, Enrgy, BusEq, Telcm, Hlth , Money, Other]
8     [NoDur, Manuf, Enrgy, BusEq, Telcm, Hlth , Mon...
9     [NoDur, Manuf, Enrgy, BusEq, Telcm, Shops, Hlt...
10    [NoDur, Manuf, Enrgy, BusEq, Telcm, Utils, Sho...
11    [NoDur, Manuf, Enrgy, Chems, BusEq, Telcm, Uti...
12    [NoDur, Durbl, Manuf, Enrgy, Chems, BusEq, Tel...
dtype: object

In [90]:
def xval_5fold(y, x , random=False, seed = 20201231,fold=5):
    # Use numpy arrays for simplicity
    y = np.asarray(y)
    x = np.asarray(x)
    n = y.shape[0]
    if random:
        # If randomisation is needed, use either the default or provided seed
        rg = np.random.default_rng(seed)
        # Generate a set of index values to use to randomly reorder the data
        # After randomisation, we can use the data as if it is is inorder!
        ind = rg.permutation(np.arange(n))
        y = y[ind]
        x = x[ind]
    # Compute the block size
    block = n / fold
    sse = 0.0
    for i in range(int(fold)):
        # Start and end of each block need to be integers or we lose an observation
        # Rounding ensures that we get all observations since int rounds down
        st = int(np.round(i * block))
        en = int(np.round((i + 1) * block))
        # Construct the indicies of the observations that we leave out
        leave_out = np.r_[st:en]
        # The included are the one that we don't leave out
        include = np.setdiff1d(np.arange(n), leave_out)
        # Compute the regression coefficients
        beta = lstsq(x[include], y[include], rcond=None)[0]
        # Compute the residuals and add to the sse
        resid = y[st:en]- x[st:en] @ beta
        sse += resid @ resid
    return sse

xval_5fold(y,x[best_models[2]],fold=10)

# Randomisation is better when we suspect the model may not be totally stable


784.7667486851252

In [92]:
bsr_sse_xv = {}
for n_var in best_models:
    bsr_sse_xv[n_var] = xval_5fold(y, x[best_models[n_var]], random=False,fold=5)
bsr_sse_xv = pd.Series(bsr_sse_xv)
bsr_model = best_models[bsr_sse_xv.idxmin()]
print(bsr_model)
bsr_sse_xv

['NoDur', 'Manuf', 'Enrgy', 'BusEq', 'Telcm', 'Shops', 'Hlth ', 'Money', 'Other']


1     1148.876144
2      771.982254
3      494.175842
4      368.090926
5      283.350797
6      235.289796
7      210.449663
8      208.251414
9      204.369481
10     208.891542
11     209.923549
12     209.984916
dtype: float64

### Exercise 43

Select the best tracking portfolio using Forward Stepwise Regression.

In [53]:
included = []

for i in range(p):
    # Excluded are the columns not already included in the model
    excluded = [col for col in x if col not in included]
    # Initialise best sse to inf so it will be beaten by some model
    best_sse = np.inf
    # Loop across excluded
    for col in excluded:
        # Add the new col to the included list and select the x data
        try_x = x[included + [col]]
        # Compute the regression and the SSE
        beta = lstsq(try_x, y, rcond=None)[0]
        resid = y - try_x @ beta
        sse = resid @ resid
        # If this SSE is better than our current best SSE, retain it and the new variable added
        if sse < best_sse:
            best_sse = sse
            next_var = col
    # Add the variable that reduced the SSE the most
    included.append(next_var)

# Builds up the list of what should be included in order
print(included)

['Other', 'BusEq', 'NoDur', 'Money', 'Enrgy', 'Telcm', 'Hlth ', 'Manuf', 'Shops', 'Utils', 'Chems', 'Durbl']


In [55]:
# Now cross validate models selected by forward stepwise and select best one
fsr_sse_sv = {}
for i in range(1,p+1):
    fsr_sse_sv[i] = xval_5fold(y,x[included[:i]])
fsr_sse_sv = pd.Series(fsr_sse_sv)
forstep_model = included[:fsr_sse_sv.idxmin()]
print(forstep_model)
fsr_sse_sv

['Other', 'BusEq', 'NoDur', 'Money', 'Enrgy', 'Telcm', 'Hlth ', 'Manuf', 'Shops']


1     1148.876144
2      785.510416
3      494.175842
4      373.038967
5      283.350797
6      235.289796
7      210.449663
8      208.251414
9      204.369481
10     208.891542
11     209.923549
12     209.984916
dtype: float64

In [121]:
# forward stepwise function: 
# requires x: x dataframe , y series, p = number of columns, requires xval function too
# input x, y, p

def forward_stepwise(x, y, p):
    included = []

    for i in range(p):
        excluded = [col for col in x if col not in included]
        best_sse = np.inf
        for col in excluded:
            try_x = x[included + [col]]
            beta = lstsq(try_x, y, rcond=None)[0]
            resid = y - try_x @ beta
            sse = resid @ resid
            if sse < best_sse:
                best_sse = sse
                next_var = col
        included.append(next_var)
    
    fsr_sse_sv = {}
    for i in range(1,p+1):
        fsr_sse_sv[i] = xval_5fold(y,x[included[:i]])
    fsr_sse_sv = pd.Series(fsr_sse_sv)
    forward_step_model = included[:fsr_sse_sv.idxmin()]
    return forward_step_model, fsr_sse_sv, [fsr_sse_sv.min(), fsr_sse_sv.idxmin()]

print(forward_stepwise(x, y , p)[0])
print()
print(forward_stepwise(x, y, p)[1])

['Other', 'BusEq', 'NoDur', 'Money', 'Enrgy', 'Telcm', 'Hlth ', 'Manuf', 'Shops']

1     1148.876144
2      785.510416
3      494.175842
4      373.038967
5      283.350797
6      235.289796
7      210.449663
8      208.251414
9      204.369481
10     208.891542
11     209.923549
12     209.984916
dtype: float64


### Exercise 44

Use Backward Stepwise Regression to select the tracking portfolio.

In [60]:
included = list(x.columns)
removed = []

for i in range(p):
    best_sse = np.inf
    for col in included:
        try_col = included[:]
        try_col.remove(col)
        try_x = x[try_col]
        beta = lstsq(try_x,y,rcond=None)[0]
        resid = y - try_x @ beta
        sse = resid @ resid
        if sse < best_sse:
           best_sse = sse
           next_drop = col
    removed.append(next_drop)
    included.remove(next_drop)

print(removed)

['Durbl', 'Chems', 'Utils', 'Shops', 'Manuf', 'Hlth ', 'Telcm', 'Other', 'Enrgy', 'NoDur', 'BusEq', 'Money']


In [67]:
included = removed[::-1]

bsr_sse_sv = {}
for i in range(1,p+1):
    bsr_sse_sv[i] = xval_5fold(y,x[included[:i]])
bsr_sse_sv = pd.Series(bsr_sse_sv)
backstep_model = included[:bsr_sse_sv.idxmin()]
print(backstep_model)
bsr_sse_sv

['Money', 'BusEq', 'NoDur', 'Enrgy', 'Other', 'Telcm', 'Hlth ', 'Manuf', 'Shops']


1     2348.988056
2      771.982254
3      528.393322
4      368.090926
5      283.350797
6      235.289796
7      210.449663
8      208.251414
9      204.369481
10     208.891542
11     209.923549
12     209.984916
dtype: float64

### Exercise 45

Using scikit-learn to cross-validate the Backward Stepwise selected models.


In [74]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

# initilize a LR that will not include an intercept (we almost never do in financial data)
lr = LinearRegression(fit_intercept=False)

# a dictionary to hold the mse
bsr_mse_cv = {}
for i in range(1, p+1):
    # get the x data to use
    included_x = x[best_models[i]]
    bsr_mse_cv[i] = cross_val_score(
        lr, included_x, y, scoring = "neg_mean_squared_error"
    ).mean()

bsr_mse_cv = pd.Series(bsr_mse_cv)
print(included[:bsr_mse_cv.idxmax()])
bsr_mse_cv

['Money', 'BusEq', 'NoDur', 'Enrgy', 'Other', 'Telcm', 'Hlth ', 'Manuf', 'Shops']


1    -2.735419
2    -1.838053
3    -1.176609
4    -0.876407
5    -0.674645
6    -0.560214
7    -0.501071
8    -0.495837
9    -0.486594
10   -0.497361
11   -0.499818
12   -0.499964
dtype: float64

### Exercise 46

Repeat the cross-validation using scikit-learn using randomly selected values in each block.

In [78]:
from sklearn.model_selection import KFold

lr = LinearRegression(fit_intercept=False)

bsr_mse_cv = {}
cv= KFold(5, shuffle=True, random_state=20201231)

for i in range(1, p+1):
    bsr_mse_cv[i] = cross_val_score(
        lr, x[included[:i]], y, scoring="neg_mean_squared_error", cv=cv
    ).mean()

bsr_mse_cv = pd.Series(bsr_mse_cv)
print(included[:bsr_mse_cv.idxmax()])
bsr_mse_cv

['Money', 'BusEq', 'NoDur', 'Enrgy', 'Other', 'Telcm', 'Hlth ', 'Manuf', 'Shops', 'Utils', 'Chems']


1    -5.436108
2    -1.799260
3    -1.193152
4    -0.759860
5    -0.574028
6    -0.461058
7    -0.406622
8    -0.379140
9    -0.369990
10   -0.371283
11   -0.369444
12   -0.380203
dtype: float64

### Exercise 47

Evaluate the models selected using the sample that was _not_ used in fitting to assess the out-of-sample performance based on SSE.

In [96]:
y_oos = vwm.loc["2015":,"VWM"]
x_oos = industries["2015":]
beta = lstsq(x[bsr_model], y, rcond=None)[0]
pred = x_oos[bsr_model] @ beta
resid_oos = y_oos - x_oos[bsr_model] @ beta
oos_sse = resid_oos @ resid_oos
print(oos_sse)

22.398513710136537


In [89]:
beta = lstsq(x[forstep_model], y, rcond=None)[0]
pred = x_oos[forstep_model] @ beta
resid_oos = y_oos - x_oos[forstep_model] @ beta
oos_sse = resid_oos @ resid_oos
oos_sse

22.398513710136463

In [100]:
def oos_sse_fun(model):
    beta = lstsq(x[forstep_model], y, rcond=None)[0]
    pred = x_oos[forstep_model] @ beta
    resid_oos = y_oos - x_oos[forstep_model] @ beta
    oos_sse = resid_oos @ resid_oos
    return oos_sse

print(oos_sse_fun(backstep_model))

22.398513710136463


### Exercise 47

Compute the in-sample and out-of-sample $R^2$ for the selected models.

The out-of-sample $R^2$ defined as

$$ 1 - \frac{SSE_{OOS}}{TSS_{OOS}} $$

where the $SSE_{OOS}$ is the SSE using the predicted values and the $TSS_{OOS}$ is the TSS computed using the in-sample value (without demeaning since the models we are fitting do not include a constant). **Note**: If a model does not contain a constant, the $TSS_{OOS}$ is _not_ demeaned.

In [123]:
oos_tss = (y_oos **2).sum()
oos_r2 = 1- oos_sse/ oos_tss
print(f"The out of sample R2 for the out of sample period is {100*oos_r2:0.1f}%.")

The out of sample R2 for the out of sample period is 98.4%.


In [126]:
from statsmodels.api import OLS

r2 = OLS(y_oos, x_oos).fit().rsquared
print(f"The in sample R2 for the out of sample period is {100*r2:0.1f}%.")

The in sample R2 for the out of sample period is 99.9%.


### Exercise 48

Use scikit-learn to produce out-of-sample fits and compute the out-of-sample SSE.  Verify this value is the same as you found previously.